In [1]:
from sklearn import metrics
import tensorflow as tf
from  read_proces_data import *
sess = tf.InteractiveSession()

In [2]:
# create lstm cell in tensorflow
hidden_units = 128
lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units =\
        hidden_units,state_is_tuple=True)

batch_size = 20
max_seq = 3260
feature_vec_len = 57
x = tf.placeholder(tf.float32, shape = [batch_size,max_seq,feature_vec_len])
s_len = tf.placeholder(tf.int32,shape=[batch_size])
# run lstm over different sequence length
output_, state = tf.nn.dynamic_rnn(\
        lstm_cell,\
        x,\
#        initial_state = i_state,\
        sequence_length=s_len,\
        dtype=tf.float32)
# take all batch, last output of each batch and full output vector
def take_subarray(array , index):
    return array[range(0,batch_size),index-1]
output = tf.py_func(take_subarray,[output_,s_len],tf.float32)
#print output
# define final output value
target_value = tf.placeholder(tf.float32, shape=[batch_size])
# define weights and bias from output of lstm cell to network final output
rv1 = tf.truncated_normal([lstm_cell.output_size, 2 * lstm_cell.output_size],\
        stddev=0.1,dtype=tf.float32)
Wo1 = tf.Variable(rv1,dtype=tf.float32)

rv2 = tf.truncated_normal([2 * lstm_cell.output_size, 1],\
        stddev=0.1,dtype=tf.float32)
Wo2 = tf.Variable(rv2,dtype=tf.float32)

rb1 = tf.constant(0.1,shape=[2 * lstm_cell.output_size],dtype=tf.float32)
b1 = tf.Variable(rb1,dtype=tf.float32)

rb2 = tf.constant(0.1,shape=[1],dtype=tf.float32)
b2 = tf.Variable(rb2,dtype=tf.float32)
# initialize variables
# calculate final output

s1  = tf.matmul(output, Wo1) + b1
o1 = tf.nn.relu(s1)
logits  = tf.matmul(o1, Wo2) + b2
observed = tf.sigmoid(logits)
#observed = tf.nn.relu(logits)

# calculate cost as calculated in logistic classification.
logistic_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(\
        labels = target_value, logits = tf.reshape(logits,[batch_size])))
#cost = tf.reduce_mean(-1.0 * target_value * tf.log(observed) - \
#        (1 - target_value) * tf.log(1 - observed))
#cost = tf.reduce_mean(-1 * target_value * tf.log(observed))
#cost = tf.reduce_mean(tf.square(target_value - observed))
# optimize the cost
train_step = tf.train.AdamOptimizer(1e-4).minimize(logistic_cost)
def threshold_fn(array):
    return array >= 0.50
obser = tf.py_func(threshold_fn,[observed], tf.bool)
observe = tf.cast(obser,tf.float32)

sess.run(tf.global_variables_initializer())

In [3]:
# find max no of times to roll back
max_seq_len = find_max_seq_len()
print max_seq_len

finding max seq len in train set
finding max seq len in test set
max s_len found
3259


In [6]:
# number of times to iterate over whole training dataset
iter_over_files =  1
for itera in range(0, iter_over_files):
    for file_ in file_dir:
        # get features from current indexed file
        features = get_train_data(file_)
        # get target values corresponding to this file
        target_values = get_target_values(file_)
        # training features is np array with max size and padding
        training_features, seq_len = prepare_data(features, max_seq_len)
        # scale features to have 0 mean and 1 variance
        scaled_features = feature_scaling(training_features, batch_size,\
                seq_len)
        # check if there is any NaN or Inf entries in data
        isnan = np.any(np.isnan(scaled_features))
        if True == isnan:
            print "nan entry found for file %s"%file_
            continue
        isinf = np.any(np.isinf(scaled_features))
        if True == isinf:
            print "inf entry found for file %s"%file_
            continue
        # find independent components from given features
        inde_features = independent_components(scaled_features, batch_size,\
                seq_len)

        for i in range(0,10):
            tr_fe, sl , tv =\
            sample_data(inde_features,target_values,seq_len,batch_size)

            sess.run(train_step, feed_dict = {target_value:tv,\
                    x:tr_fe, s_len:sl})

            if i%9 == 0:
                print "iteration %d"%i
                print "cost %f"%logistic_cost.eval(feed_dict = {target_value:tv,\
                    x:tr_fe, s_len:sl})

                y = observe.eval(feed_dict = {x:tr_fe, s_len:sl})
#        print tv
#        print y
                f1s = metrics.f1_score(tv,y)
                print "f1-score is %f"%f1s
                fpr, tpr, thresholds = metrics.roc_curve(tv, y, pos_label=1)
                auc = metrics.auc(fpr,tpr)
                print "area under curve is %f"%auc
                sk_accuracy = metrics.accuracy_score(tv,y)
                print "accuracy %f"%sk_accuracy
                print " "
                
                # evaluate over whole file
                y = np.zeros(0)
                if "05" in file_:
                    print file_
                    for index in range(0,100,batch_size):
                        yfile = observe.eval(feed_dict = {\
                            x:inde_features[index:index+batch_size], \
                            s_len:seq_len[index:index+batch_size]})
                        y = np.append(y,yfile)
                else:
                    for index in range(0,100,batch_size):
                        yfile = observe.eval(feed_dict = {\
                            x:inde_features[index:index+batch_size], \
                            s_len:seq_len[index:index+batch_size]})
                        y = np.append(y,yfile)

                f1score = metrics.f1_score(target_values,y)
                print "f1-score of file is %f"%f1score
                fprr, tprr, thresholds = metrics.roc_curve(target_values, y, pos_label=1)
                aucc = metrics.auc(fprr,tprr)
                print "area under curve of file is %f"%aucc
                sk_acc = metrics.accuracy_score(target_values,y)
                print "accuracy of file is %f"%sk_acc
                print " "

iteration 0
cost 0.713331
f1-score is 0.642857
area under curve is 0.500000
accuracy 0.500000
 


ValueError: Cannot feed value of shape (0, 3260, 57) for Tensor u'Placeholder:0', which has shape '(20, 3260, 57)'